In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [7]:
df_new=pd.read_csv('/content/gdrive/MyDrive/MainQuest04/step02.csv')

In [16]:
customer_category_counts = df_new.groupby(['customer_unique_id', 'product_category_name']).size().reset_index(name='purchase_count')

#여러 개의 카테고리를 구매한 고객만 필터링
multi_category_customers = customer_category_counts[customer_category_counts.duplicated(subset=['customer_unique_id'], keep=False)]

#각 고객이 구매한 모든 카테고리 조합을 생성
customer_category_pairs = multi_category_customers.groupby('customer_unique_id')['product_category_name'].apply(set).reset_index()

#모든 조합을 추출하여 빈도수 계산
from itertools import combinations
from collections import Counter

category_combinations = Counter()

for categories in customer_category_pairs['product_category_name']:
    for combo in combinations(categories, 2):
        category_combinations[frozenset(combo)] += 1

#데이터프레임으로 변환
category_combinations_df = pd.DataFrame(category_combinations.items(), columns=['Category Pair', 'Count'])
category_combinations_df = category_combinations_df.sort_values(by='Count', ascending=False)

#'bed_bath_table'과 함께 구매된 카테고리 필터링
bed_bath_related_combinations = category_combinations_df[category_combinations_df['Category Pair'].apply(lambda x: 'bed_bath_table' in x)].copy()

#'bed_bath_table'과 함께 구매된 카테고리를 추출
bed_bath_related_combinations['Other Category'] = bed_bath_related_combinations['Category Pair'].apply(lambda x: list(x - {'bed_bath_table'})[0])

#'bed_bath_table'의 총 구매 횟수 계산
bed_bath_total_sales = df_new[df_new['product_category_name'] == 'bed_bath_table'].shape[0]

#전환율(%) 계산
bed_bath_related_combinations['Conversion Rate (%)'] = (bed_bath_related_combinations['Count'] / bed_bath_total_sales) * 100

#가장 높은 전환율을 가진 카테고리 찾기
top_conversion_category = bed_bath_related_combinations.sort_values(by='Count', ascending=False).iloc[0]

#최종 결과 출력
top_conversion_category
bed_bath_related_combinations.head() # 여기서 말하는 전환률은 함께 구입한 경우의 수를 의미한다.

,Category Pair,Count,Other Category,Conversion Rate (%)
24,"(bed_bath_table, furniture_decor)",156,furniture_decor,1.403509
10,"(bed_bath_table, housewares)",67,housewares,0.602789
33,"(bed_bath_table, home_confort)",54,home_confort,0.485830
45,"(baby, bed_bath_table)",37,baby,0.332883
23,"(bed_bath_table, health_beauty)",29,health_beauty,0.260909


In [19]:


# 1. 구매 이력 정리 (고객별 구매 순서 정렬)
if 'order_purchase_timestamp' in df_new.columns:
    df_new['order_purchase_timestamp'] = pd.to_datetime(df_new['order_purchase_timestamp'])
    df_sorted = df_new.sort_values(by=['customer_unique_id', 'order_purchase_timestamp'])
else:
    # 구매 날짜 정보가 없는 경우, order_id 기준 정렬
    df_sorted = df_new.sort_values(by=['customer_unique_id', 'order_id'])

# 2. 고객별 첫 번째 구매 카테고리 확인
df_sorted['previous_category'] = df_sorted.groupby('customer_unique_id')['product_category_name'].shift(1)

# 3. 'bed_bath_table' 구매 이후 다른 카테고리를 구매한 경우 필터링
df_transitions = df_sorted[(df_sorted['previous_category'] == 'bed_bath_table') &
                           (df_sorted['product_category_name'] != 'bed_bath_table')]

# 4. bed_bath_table 이후 가장 많이 구매된 카테고리 및 전환율 계산
transition_counts = df_transitions['product_category_name'].value_counts()
total_bed_bath_customers = df_sorted[df_sorted['product_category_name'] == 'bed_bath_table']['customer_unique_id'].nunique()

# 5. 전환율(%) 계산
transition_rates = (transition_counts / total_bed_bath_customers) * 100
transition_rates_df = transition_rates.reset_index()
transition_rates_df.columns = ['Next Category', 'Transition Rate (%)']

# 결과 출력
transition_rates_df

,Next Category,Transition Rate (%)
0,furniture_decor,0.940405
1,housewares,0.349918
2,home_confort,0.306178
3,health_beauty,0.185894
4,baby,0.164024
5,cool_stuff,0.131219
6,computers_accessories,0.109349
7,garden_tools,0.087479
8,sports_leisure,0.087479
9,pet_shop,0.087479
